# Анализ базы данных книг и отзывов/обзоров на них

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель проекта:** на основе информации в базе данных сформировать список наиболее популярных авторов книг и издательств, которые занимаются выпуском книг, а так же определение среднего количества обзоров от поьзователей с целью дальнейшего формирования предложения для пользователей и прогнозирования их активности.

**План проекта:**
1. Получение общей информации о даных таблицах (просмотр первых строк).
2. Исследовательсткий анализ даных:

    2.1. Анализ количества выпущенных книг после 1 января 2000 года.
    
    2.2. Анализ книг и определение количества обзоров и средней оценки;
    
    2.3. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.
    
    2.4. Определение автора с самой высокой средней оценкой книг (в анализе не участвуют книги с оценками менее 50)
    
    2.5. Определение среднего количества обзоров от пользователей (в анализе не участвуют пользователи с количеством оценок менее 50)
    
3. Выводы

### 1. Получение общей информации о данных в таблицах

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
# создадим список с названием таблиц базы данных
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']
tables

['books', 'authors', 'publishers', 'ratings', 'reviews']

In [5]:
def my_func(query):
    return pd.io.sql.read_sql(query, con = engine)

In [6]:
for i in tables:
    query = f'''
                SELECT *
                FROM {i};
    ''' 
    print('Количество строк в таблице {} - {}. Выведем первые 5 строк'
          .format(i, len(my_func(query))))

    query = f'''
                SELECT *
                FROM {i}
                LIMIT 5;
    ''' 
    display(my_func(query))
    print('___' * 15)
    print('\n')

Количество строк в таблице books - 1000. Выведем первые 5 строк


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


_____________________________________________


Количество строк в таблице authors - 636. Выведем первые 5 строк


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


_____________________________________________


Количество строк в таблице publishers - 340. Выведем первые 5 строк


...

_____________________________________________


Количество строк в таблице ratings - 6456. Выведем первые 5 строк


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


_____________________________________________


Количество строк в таблице reviews - 2793. Выведем первые 5 строк


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


_____________________________________________




**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [7]:
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT 
    table_name, 
    column_name, 
    data_type, 
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS 
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


## 2. Исследовательсткий анализ даных

### 2.1. Анализ количества выпущенных книг после 1 января 2000 года.

In [8]:
query = '''
            SELECT COUNT(book_id) AS count_books
            FROM books
            WHERE publication_date > '2000-01-01'
'''
my_func(query)

,count_books
0,819


**Вывод:** 819 книг было выпущено после 1 января 2000 г.

### 2.2. Анализ книг и определение количества обзоров и средней оценки

In [9]:
query = '''
            SELECT b.title, 
                COUNT(DISTINCT re.review_id) AS count_reviews, 
                ROUND(AVG(r.rating),2) AS avg_rating
            FROM books AS b
                LEFT JOIN ratings AS r ON r.book_id = b.book_id
                LEFT JOIN reviews AS re ON re.book_id = b.book_id
            GROUP BY b.title
            ORDER BY count_reviews DESC, avg_rating DESC;
'''
my_func(query)

,title,count_reviews,avg_rating
0,Memoirs of a Geisha,8,4.14
1,Twilight (Twilight #1),7,3.66
2,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
4,The Book Thief,6,4.26
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.00
995,Essential Tales and Poems,0,4.00
996,Leonardo's Notebooks,0,4.00
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Наиболее популярная книга по количеству обзоров Memoirs of a Geisha - 8 обзоров, средний рейтинг - 4,14 балла.

На втором месте по количеству обзоров книга Twilight (Twilight #1) (7 обзоров) при это средний рейтинг 3,66 балла.

Третье место делят несколько книг по количеству обзоров (по 6 обзоров у каждой), отметим только тех, кто входит в ТОП-5:
* третье и четвертое место заняли книги про Harry Potter - Harry Potter and the Prisoner of Azkaban со средним рейтингом 4,41 балла и Harry Potter and the Chamber of Secrets со средним рейтингом 4,29 балла
* пятое место заняла книга The Book Thief со средним рейтингом 4,26 балла.

На последнем месте находится книга The Natural Way to Draw - 0 отзывов и средний рейтинг 3 балла.

### 2.3.  Определение издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
query = '''
            SELECT p.publisher, COUNT(b.book_id) AS count_books
            FROM books AS b
                LEFT JOIN publishers AS p ON p.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(b.book_id) DESC
            LIMIT 1;
'''
my_func(query)

,publisher,count_books
0,Penguin Books,42


**Вывод:** 
Издательство под названием Penguin Books выпустило наибольшее число книг (42) толще 50 страниц. 

### 2.4. Определение автора с самой высокой средней оценкой книг (в анализе не участвуют книги с оценками менее 50)

При выполнении этого задания будем придерживаться следующей последовательности:
1. Определение списка книг, на которые имеется более 50 оценок - делаем подзапрос.
2. В основном запросе будем работать только с теми книгами, которые получили в пунке 1.
3. Присоединим к таблице books таблицы authors и ratings.
4. Сгруппируем строки по имени автора и высчитаем среднюю оценку книг автора.
5. Произведем сортировку по среднему рейтингу книг автора в порядке убывания среднего рейтинга.
6. Выведем 1 строку - имя автора и максимальное значение среднего рейтинга книг автора.

In [11]:
query = '''
            SELECT a.author, ROUND(AVG(r.rating), 2) AS avg_rating
            FROM books AS b
                LEFT JOIN authors AS a ON a.author_id = b.author_id
                LEFT JOIN ratings AS r ON r.book_id = b.book_id
            WHERE b.book_id IN (SELECT book_id
                                FROM ratings
                                GROUP BY book_id
                                HAVING COUNT(book_id) > 50)
            GROUP BY a.author
            ORDER BY AVG(r.rating) DESC
            LIMIT 1;
'''
my_func(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


**Вывод**:
Автор книг J.K. Rowling/Mary GrandPré имеет наибольшую среднюю оценку (4,29) своих произведений среди других авторов.

### 2.5. Определение среднего количества обзоров от пользователей (в анализе не участвуют пользователи с количеством оценок менее 50)

При выполнении этого задания будем придерживаться следующей последовательности:
1. Создадим временную таблицу temp, в которой будет храниться информация о пользователях и количество оставленных ими обзоров. При этом в эту таблицу не войдут пользователи оставившие меньше 50 оценок книг.
2. На основании данных временной таблицы посчитаем среднее количество обзоров пользователей.

In [12]:
query = '''
            WITH temp AS (SELECT username, 
                                COUNT(username) AS count_rev
                            FROM reviews
                            WHERE username IN (SELECT username
                                                FROM ratings
                                                GROUP BY username
                                                HAVING COUNT(username) > 50)
                            GROUP BY username)
                
            SELECT ROUND(AVG(count_rev), 2) AS avg_rev
            FROM temp;
'''
my_func(query)

,avg_rev
0,24.33


**Вывод:** 
В среднем 24 обзора делает пользователь на книги.

### Дополнительный анализ

Выведите таблицу, которая будет содержать по году публикации:

- количество издательств,
- количество выпущенных книг
- сколько всего тысяч страниц было в изданных книгах

(отобразить только те года, в которых издано более 30 книг)

In [13]:
query = '''
            SELECT EXTRACT(YEAR FROM publication_date) AS year,
                    COUNT(DISTINCT publisher) AS count_publisher, 
                    COUNT(DISTINCT title) AS count_book, 
                    SUM(num_pages) AS total_page
            FROM publishers AS p
                LEFT JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE EXTRACT(YEAR FROM publication_date) IN (SELECT DISTINCT EXTRACT(YEAR FROM publication_date) AS year_publications
                                                        FROM books
                                                        GROUP BY EXTRACT(YEAR FROM publication_date)
                                                        HAVING COUNT(title) > 30)
            GROUP BY 1;
'''
my_func(query)

,year,count_publisher,count_book,total_page
0,1999.0,26,41,15763
1,2000.0,35,38,13328
2,2001.0,41,60,21758
3,2002.0,62,94,38597
4,2003.0,65,105,41423
5,2004.0,88,124,46779
6,2005.0,89,138,55967
7,2006.0,109,184,68302
8,2007.0,38,50,18258


In [14]:
my_func(query).T

,0,1,2,3,4,5,6,7,8
year,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0
count_publisher,26.0,35.0,41.0,62.0,65.0,88.0,89.0,109.0,38.0
count_book,41.0,38.0,60.0,94.0,105.0,124.0,138.0,184.0,50.0
total_page,15763.0,13328.0,21758.0,38597.0,41423.0,46779.0,55967.0,68302.0,18258.0


**Вывод**: с 1999 по 2007 гг. было выпущенно более 30 книг в год. При этом наблюдается ежегодная динамика к увеличению количества выпускаемых книг и рост издательств.

В 2007 г. видно падение по данным показателям, при этом необходим дополнительный анализ, где необходимо перепроверить на сколько полны данные по этому году.

Выведите в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек и отдельно среднюю оценку остальных книг, сделайте выводы какой рейтинг больше

In [15]:
query = '''
            WITH temp AS (
                        SELECT ROUND(AVG(rating), 3) AS avg_rating_3
                        FROM books AS b
                            LEFT JOIN ratings AS r ON b.book_id = r.book_id
                        WHERE title IN (SELECT DISTINCT title
                                        FROM books AS b
                                            LEFT JOIN reviews AS r ON b.book_id = r.book_id
                                        GROUP BY b.title
                                        HAVING COUNT(r.username) > 3)
                        ),
                        
            temp_2 AS (
                        SELECT ROUND(AVG(rating),3) 
                                AS avg_rating_all
                        FROM ratings AS b
                    )
                
            SELECT avg_rating_3, avg_rating_all
            FROM (SELECT *, ROW_NUMBER() OVER(ORDER BY avg_rating_3) n 
                    FROM temp) AS a
                FULL OUTER JOIN (SELECT avg_rating_all,
                                        ROW_NUMBER() OVER(ORDER BY avg_rating_all) n
                                FROM temp_2) AS b
                ON a.n = b.n
                
'''
my_func(query)

,avg_rating_3,avg_rating_all
0,3.949,3.928


**Вывод**: Средний рейтинг книг с количеством отзывов более 3 незначительно больше общего среднего рейтинга, разница составляет 0,021 балла.

### 3. Общий вывод по проекту

Для анализа была предоставлена база данных, которая содержала в сете следующие таблицы:
* **Таблица `books`** содержит данные о книгах (Количество строк в таблице books - 1000),
* **Таблица `authors`** содержит данные об авторах (Количество строк в таблице authors - 636),
* **Таблица `publishers`** содержит данные об издательствах (Количество строк в таблице publishers - 340),
* **Таблица `ratings`** содержит данные о пользовательских оценках книг (Количество строк в таблице ratings - 6456),
* **Таблица `reviews`** содержит данные о пользовательских обзорах на книги (Количество строк в таблице reviews - 2793).

**Результаты исследовательского анализа:**
* 819 книг было выпущено после 1 января 2000 г.
* Топ-5 популярных книги: 
    - Наиболее популярная книга по количеству обзоров Memoirs of a Geisha - 8 обзоров, средний рейтинг - 4,14 балла.
    - На втором месте по количеству обзоров книга Twilight (Twilight #1) (7 обзоров) при это средний рейтинг 3,66 балла.
    - Третье место делят несколько книг по количеству обзоров (по 6 обзоров у каждой), отметим только тех, кто входит в ТОП-5:
        * третье и четвертое место заняли книги про Harry Potter - Harry Potter and the Prisoner of Azkaban со средним рейтингом 4,41 балла и Harry Potter and the Chamber of Secrets со средним рейтингом 4,29 балла
        * пятое место заняла книга The Book Thief со средним рейтингом 4,26 балла.
    - На последнем месте находится книга The Natural Way to Draw - 0 отзывов и средний рейтинг 3 балла.
* Издательство под названием Penguin Books выпустило наибольшее число книг (42) толще 50 страниц.
* Автор книг J.K. Rowling/Mary GrandPré имеет наибольшую среднюю оценку (4,29) своих произведений среди других авторов.
* В среднем 24 обзора делает пользователь на книги.
* с 1999 по 2007 гг. было выпущенно более 30 книг в год. При этом наблюдается ежегодная динамика к увеличению количества выпускаемых книг и рост издательств. В 2007 г. видно падение по данным показателям, при этом необходим дополнительный анализ, где необходимо перепроверить на сколько полны данные по этому году.
* Средний рейтинг книг с количеством отзывов более 3 незначительно больше общего среднего рейтинга, разница составляет 0,021 балла.